In [1]:
# import dataset as matrices
#define key values
import numpy as np
import pandas as pd
from ortools.graph import pywrapgraph
import time

df_wish = np.array(pd.read_csv("../input/child_wishlist_v2.csv", header=None))[:, 1:]
df_good = np.array(pd.read_csv("../input/gift_goodkids_v2.csv", header=None))[:, 1:]
number_gifts = 1000
number_gift_types = 1000
number_children = 1000000
number_wishes = 100
number_goodkids = 1000

df_good[:2]

In [2]:

start_nodes = []
end_nodes = []
capacities = []
costs = []
supplies = []

for gift in range(number_gift_types):
    start_nodes.append(gift)
    end_nodes.append(1000)
    capacities.append(number_gifts)
    costs.append(0)
    supplies.append(number_gifts)
    
supplies.append(0)

for child in range(number_children):
    start_nodes.append(1000)
    end_nodes.append(child+1001)
    capacities.append(1)
    costs.append(1)
    if child<=5000:
        if child%3 == 0:
            supplies.append(-3)
        else:
            supplies.append(0)
    elif child >= 5001 and child <=45000:
        if child%2 == 1:
            supplies.append(-2)
        else:
            supplies.append(0)
    else:
        supplies.append(-1)
        
    
    
    
for key in range(number_children):
    if key%10000 == 0:
        print((key/(len(df_wish)))*100,'%')
    for key_g in range(75):
        gift = df_wish[key, key_g]
        start_nodes.append(gift)
        end_nodes.append(1001+key)
#         capacities.append(1)
        key_c = np.where(df_good[gift] == key)[0]
#         for key_c, kid in enumerate(df_good.iloc[gift]):
        if key<=5000:
            capacities.append(3)
            if key_c:
                costs.append((-200 * (number_wishes - key_g)) + (-20.0 * (number_goodkids - int(key_c))))
            else:
                costs.append(-200 * (number_wishes - key_g))
        elif key >= 5001 and key <=45000:
            capacities.append(2)
            if key_c:
                costs.append((-100 * (number_wishes - key_g)) + (-10.0 * (number_goodkids - int(key_c))))
            else:
                costs.append(-100 * (number_wishes - key_g))
        else:
            capacities.append(1)
            if key_c:
                costs.append((-2 * (number_wishes - key_g)) + (-0.2 * (number_goodkids - int(key_c))))
            else:
                costs.append(-2 * (number_wishes - key_g))

            

    
            
print('graph completed')        
# print(costs)
                
    



In [ ]:
# Instantiate a SimpleMinCostFlow solver.
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

# Add each arc.
for i in range(0, len(start_nodes)):
    min_cost_flow.AddArcWithCapacityAndUnitCost(int(start_nodes[i]), int(end_nodes[i]), int(capacities[i]), int(costs[i]))

# Add node supplies.

for i in range(0, len(supplies)):
    min_cost_flow.SetNodeSupply(i, supplies[i])

In [ ]:

gift_sharing = min_cost_flow.SolveMaxFlowWithMinCost()
print('Solved with status', gift_sharing, ' Pull results....')
print('Maximum flow:', min_cost_flow.MaximumFlow(), ' = ', number_children)

In [ ]:
# Solve minimum cost flow problem on graph input and get result_list of gift-child pair

result_list = []
for i in range(min_cost_flow.NumArcs()):
    if i % 5000000 == 0:
        print(i)
    cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
    if cost != 0:
        re_tuple = (min_cost_flow.Tail(i),(min_cost_flow.Head(i)-1001))
        result_list.append(re_tuple)


In [ ]:
for result in result_list:
    if result[1]<= 5000:
        if result[1]%3 == 0:
            tup1=(result[0], result[1]+1)
            tup2=(result[0], result[1]+2)
            result_list.append(tup1)
            result_list.append(tup2)
    if result[1]>= 5001 and result[1]<= 45000:
        if result[1]%2 == 1:
            tup=(result[0], result[1]+1)
            result_list.append(tup)


In [ ]:
for result in result_list:
    if result[1]<100:
        print(result)

In [ ]:
# Evaluate the result (Happiness)

total_child_happiness = 0
nch = 0
total_santa_happiness = 0
nsh = 0

for result in result_list:
#     child_happiness = 0
    try:
        ch = np.where(df_wish[result[1]] == result[0])[0]
        child_happ = 2*(100 - int(ch))
    except:
        child_happ = -1
    try:
        sh = np.where(df_good[result[0]] == result[1])[0]
        santa_happ = 2*(1000 - int(sh))
    except:
        santa_happ = -1
    child_happiness = child_happ/200
    santa_happiness = santa_happ/2000
    total_child_happiness = total_child_happiness + child_happiness
    
    total_santa_happiness = total_santa_happiness + santa_happiness


nch = total_child_happiness/number_children
nsh = total_santa_happiness/(number_gift_types*number_gifts)
        
print("NCH: ",nch)
print("NSH: ",nsh)
print("total_happiness: ",(nch ** 3)+(nsh ** 3))

print("done")        
 

In [ ]:
# Get lists of unassigned gifts and unassigned children
from collections import Counter
count_list = []
new_gift_list = []
new_children_list = []
for result in result_list:
    if result[0] != 1000:
        count_list.append(result[0])
    else:
        new_children_list.append(result[1])
count = Counter(count_list)
for item, cnt in count.items():
    if cnt < 1000:
        tup = (item, 1000-cnt)
        new_gift_list.append(tup)
len(new_children_list)
# new_gift_list   
new_children_list

In [ ]:
# Create graph of remaining gifts and unassigned children
start_nodes2 = []
end_nodes2 = []
capacities2 = []
costs2 = []
supplies2 = []


for gift in range(len(new_gift_list)):
    for k, child in enumerate(new_children_list):
        start_nodes2.append(gift)
        end_nodes2.append(len(new_gift_list)+k)
        capacities2.append(1)
        for good in df_good:
            cc = np.where(df_good[new_gift_list[gift][0]] == child)[0]
            if cc:
                costs2.append(-2*(number_gift_types-int(cc)))
            else:
                costs2.append(1)
    supplies2.append(new_gift_list[gift][1])
    
for i in range(len(new_children_list)):
    supplies2.append(-1)
    

    
    


In [ ]:
# solve new graph

# Instantiate a SimpleMinCostFlow solver.
min_cost_flow2 = pywrapgraph.SimpleMinCostFlow()

# Add each arc.
for i in range(0, len(start_nodes2)):
    min_cost_flow2.AddArcWithCapacityAndUnitCost(int(start_nodes2[i]), int(end_nodes2[i]), int(capacities2[i]), int(costs2[i]))

# Add node supplies.

for i in range(0, len(supplies2)):
    min_cost_flow2.SetNodeSupply(i, supplies2[i])

In [ ]:

gift_sharing2 = min_cost_flow2.SolveMaxFlowWithMinCost()
print('Solved with status', gift_sharing2, ' Pull results....')
print('Maximum flow:', min_cost_flow2.MaximumFlow(), ' = ', len(new_children_list))



In [ ]:
# solve new graph
result_list2 = []
for i in range(min_cost_flow2.NumArcs()):
    if i % 5000000 == 0:
        print(i)
    cost2 = min_cost_flow2.Flow(i) * min_cost_flow2.UnitCost(i)
#     if min_cost_flow.UnitCost(i) == 1:
#         print(i)
    if cost2 != 0:
        for k1,v1 in enumerate(new_gift_list):
            if k1 == min_cost_flow2.Tail(i):
                ggift = v1[0]
                cchild_k = min_cost_flow2.Head(i) - len(new_gift_list)
                for k2,v2 in enumerate(new_children_list):
                    if k2 == cchild_k:
                        cchild = v2
        re_tuple2 = (ggift,cchild)
        result_list2.append(re_tuple2)

In [ ]:
#update result_list
for index, result in enumerate(result_list):
    for result2 in result_list2:
        if result[1] == result2[1]:
            print(result)
            result_list[index] = result2

In [ ]:
# Re-evaluate happiness

total_child_happiness = 0
nch = 0
total_santa_happiness = 0
nsh = 0

for result in result_list:
    try:
        ch = np.where(df_wish[result[1]] == result[0])[0]
        child_happ = 2*(100 - int(ch))
    except:
        child_happ = -1
    try:
        sh = np.where(df_good[result[0]] == result[1])[0]
        santa_happ = 2*(1000 - int(sh))
    except:
        santa_happ = -1
    child_happiness = child_happ/200
    santa_happiness = santa_happ/2000
    total_child_happiness = total_child_happiness + child_happiness
    
    total_santa_happiness = total_santa_happiness + santa_happiness


nch = total_child_happiness/number_children
nsh = total_santa_happiness/(number_gift_types*number_gifts)
        
print("NCH: ",nch)
print("NSH: ",nsh)
print("total_happiness: ",(nch ** 3)+(nsh ** 3))


print("done")        